In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
features = ['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Pclass']
x = df[features]
y = df['Survived']


In [4]:
x.describe()

,Age,SibSp,Parch,Fare,Pclass
count,714.000000,891.000000,891.000000,891.000000,891.000000
mean,29.699118,0.523008,0.381594,32.204208,2.308642
std,14.526497,1.102743,0.806057,49.693429,0.836071
min,0.420000,0.000000,0.000000,0.000000,1.000000
25%,20.125000,0.000000,0.000000,7.910400,2.000000
50%,28.000000,0.000000,0.000000,14.454200,3.000000
75%,38.000000,1.000000,0.000000,31.000000,3.000000
max,80.000000,8.000000,6.000000,512.329200,3.000000


In [5]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Sex       891 non-null    object 
 1   Age       714 non-null    float64
 2   SibSp     891 non-null    int64  
 3   Parch     891 non-null    int64  
 4   Fare      891 non-null    float64
 5   Embarked  889 non-null    object 
 6   Pclass    891 non-null    int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [6]:
x.isnull().values.any()

True

In [7]:
x['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
x['Embarked'].fillna(value='S',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [9]:
x['Age'].fillna(x['Age'].mean(),inplace=True)
x['Fare'].fillna(x['Fare'].mean(),inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [10]:
x['Sex'] = x['Sex'].replace(['female','male'],[0,1]).astype(int)
x['Embarked'] = x['Embarked'].replace(['S','Q','C'],[1,2,3]).astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
from sklearn import preprocessing
columns = x.columns
scaler = preprocessing.StandardScaler()
std_x = scaler.fit_transform(x)
std_x = pd.DataFrame(std_x, columns=columns)
std_x.head()

,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass
0,0.737695,-0.592481,0.432793,-0.473674,-0.502445,-0.585954,0.827377
1,-1.355574,0.638789,0.432793,-0.473674,0.786845,1.942303,-1.566107
2,-1.355574,-0.284663,-0.474545,-0.473674,-0.488854,-0.585954,0.827377
3,-1.355574,0.407926,0.432793,-0.473674,0.420730,-0.585954,-1.566107
4,0.737695,0.407926,-0.474545,-0.473674,-0.486337,-0.585954,0.827377


In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(std_x, y, test_size=0.20, random_state=0)

In [13]:
x_train = np.array(x_train).T
y_train = np.array(y_train)
#y_train = np.reshape(y_train[0], 1)
x_test = np.array(x_test).T
y_test = np.array(y_test)
#y_test = np.reshape(y_test[0], 1)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((7, 712), (712,), (7, 179), (179,))

In [14]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))    
    return s

In [15]:
w = np.zeros((x_train.shape[0], 1))
b = 0
assert(isinstance(b, float) or isinstance(b, int))

In [16]:
def propagate(w, b, X, Y):
    m = X.shape[1]

    a = sigmoid(np.dot(w.T, X) + b)                                  
    cost = (-1.0/m) * (np.sum(Y*np.log(a) + (1-Y)*np.log(1-a)))                               

    dw = (1.0/m) * np.dot(X, (a-Y).T)
    db = (1.0/m) * np.sum(a-Y)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())

    return dw, db, cost

In [17]:
def optimize(w, b, X, Y, iterations, alpha, print_cost = False):

    costs = []
    
    for i in range(iterations):
        dw, db, cost = propagate(w, b, X, Y)

        w = w - alpha * dw 
        b = b - alpha * db

        if i % 100 == 0:
            costs.append(cost)
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return w, b, dw, db, costs

In [18]:
def predict(w, b, X):

    m = X.shape[1]
    y_pred = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)

    a = sigmoid(np.dot(w.T, X) + b)

    for i in range(a.shape[1]):
        if a[0, i] >= 0.5:
          y_pred[0, i] = 1 
        else:
          y_pred[0, i] = 0

    assert(y_pred.shape == (1, m))

    return y_pred

In [19]:
w, b, dw, db, costs = optimize(w, b, x_train, y_train, iterations=1000, alpha=0.001, print_cost=False)
    
y_pred = predict(w, b, x_test)

print("Accuracy: {} %".format(100 - np.mean(np.abs(y_pred - y_test)) * 100))


Accuracy: 79.88826815642457 %
